In [82]:
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import datetime
import svg_render_helpers as rdh

### file hierarchy and database connection vars

In [83]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://stanford:' + pswd + '@127.0.0.1')
db = conn['kiddraw']

### Render out images from museum station -- takes a *long* time.

#### Grab some subset of desired sketches to render

In [84]:
iteration_name = 'CDM_photodraw_e2'
this_collection = db[iteration_name]

# sessions_to_render =  this_collection.find({'$and': [{'dataType':'finalImage'}]}).distinct('sessionId')
all_sessions_this_coll = this_collection.find({'category':'this shape'}).distinct('sessionId')

print('we have {} unique kids'.format(len(all_sessions_this_coll)))


we have 161 unique kids


In [85]:
# post filtering
meta_data = pd.read_csv('final_merged_metadata.csv')
is_CDM = meta_data['site']=="CDM"
CDM_meta = meta_data[is_CDM]
valid_sessions = np.unique(CDM_meta['session_id'])

sessions_to_render = pd.DataFrame(valid_sessions)
sessions_to_render.columns = ['session_id']
print('we have {} unique kids'.format(len(sessions_to_render)))


we have 121 unique kids


#### And where are we rendering these skeches?

In [86]:
## And where are we rendering these sketches?
analysis_dir = os.getcwd()
sketch_dir = os.path.join(analysis_dir,'all_tracings')
if not os.path.exists(sketch_dir):
    os.makedirs(sketch_dir)

In [87]:
s_full = 'CDM_' + s
s_full

'CDM_CDM_photodraw_e21556311534736'

In [96]:
category_short = imrec['category'].split("this ")[1]
category_short

u'square'

In [97]:
reload(rdh)
really_run_this=1
tracing = ['this square', 'this shape']
# canvas_size = 809
# imsize = 400

canvas_size = 809
imsize = 400

if really_run_this:
   
    for s_abbrev in sessions_to_render['session_id']:
        s = 'CDM_' + s_abbrev
        image_recs = this_collection.find({'$and': [{'sessionId':s}, {'dataType':'finalImage'}, {'category':{"$in": tracing}}]}).sort('startTrialTime')    

        ## get survey data for this kid (if it exists), use to exclude
        for imrec in image_recs:                                                            
            category_short = imrec['category'].split("this ")[1]
            category_dir = os.path.join(sketch_dir,category_short)
            if not os.path.exists(category_dir):
                os.makedirs(category_dir)

            stroke_recs = this_collection.find({'$and': [{'sessionId':s}, 
                                      {'dataType':'stroke'},
                                      {'category': imrec['category']}]}).sort('startTrialTime')   
            _svg = [] # this keeps track of the strokes from THIS final image
            for strec in stroke_recs:
                _svg.append(strec['svg'])

            # don't do adults for now or blank images
            if stroke_recs.count()>0 and stroke_recs.count()<50: 
                # render out images
                try:

                    ## now get me some verts and codes!
                    Verts, Codes = rdh.get_verts_and_codes(_svg)
                    outpath = os.path.join(category_dir,'{}_{}.png'.format(imrec['sessionId'],category_short))

                    ## now render out your cumulative sketches and save out as pngs!
                    rdh.render_and_save(Verts,
                                    Codes,
                                    outpath,
                                    line_width=5,
                                    imsize=imsize,
                                    canvas_size=canvas_size)

                except Exception as e:
                    print 'Oops, something went wrong! Here is the error:'  
                    print e
                    pass